In [1]:
import datetime
import pandas as pd
import numpy as np
# The TA file was done once without the sales filters

In [8]:
from haversine import haversine
haversine([34.0037309,-81.2094413],[33.992752,-81.231010])*1000


2333.2104235433694

In [10]:
latest_TA=pd.read_excel("/home/jian/Projects/Smoothie_King/weekly_TA_updates/output_20190225/SmoothieKing_TA_of_3_miles_zips_JL_2019-03-01.xlsx",dtype=str,sheetname='zip_TA')
latest_TA=latest_TA[['TA','zip']].drop_duplicates()


In [11]:
DMA_zip=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1,usecols=[0,2])
DMA_zip.columns=['zip_cd',"DMA"]
DMA_zip=DMA_zip.drop_duplicates()
DMA_dallas_zip=DMA_zip[DMA_zip['DMA']=="DALLAS-FT. WORTH"]

In [12]:
latest_TA_Dallas=latest_TA[latest_TA['zip'].isin(DMA_dallas_zip['zip_cd'].unique().tolist())]
Dallas_TAs=latest_TA_Dallas['TA'].unique().tolist()

In [13]:
Dallas_TA_whole_zips=latest_TA[latest_TA['TA'].isin(Dallas_TAs)]
Dallas_TA_whole_zips=Dallas_TA_whole_zips.rename(columns={"zip":"zip_cd"})

In [14]:
Dallas_TA_whole_zips=pd.merge(Dallas_TA_whole_zips,DMA_dallas_zip,on="zip_cd",how="left")

In [15]:
Dallas_TA_whole_zips['DMA'].unique()

# All the TAs associated with Dallas, their zips are all in Dallas DMA

array(['DALLAS-FT. WORTH'], dtype=object)

In [16]:
latest_TA_Zips=latest_TA[['zip']].drop_duplicates()

latest_TA_Zips['in_dallas']=np.where(latest_TA_Zips['zip'].isin(Dallas_TA_whole_zips['zip_cd']),"DALLAS-FT. WORTH","Others")

In [17]:
writer=pd.ExcelWriter("/home/jian/Projects/Smoothie_King/Dallas_Map/SK_TA_zips_in_Dallas_as_0225_Full_TAfile_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
latest_TA_Zips=latest_TA_Zips.sort_values(['in_dallas','zip'])
latest_TA_Zips.to_excel(writer,"Dallas_TA_zips",index=False)
writer.save()